In [19]:
graph_disease(["Extravasation of Contrast Media", "Intracranial Hematoma, Traumatic"], view = "c")

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'font-famil…

In [2]:
# загрузка необходимых пакетов

import pandas as pd
import networkx
import matplotlib.pyplot as plt
import numpy as np
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.plotting import output_file
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

import ipycytoscape
import ipywidgets
import json

In [16]:
# загрузка необходимых таблиц

dis = pd.read_csv("diseases.tsv", sep = "\t")

In [13]:
def graph_disease(diseases, table = dis, view = "b"):
    # функция строит граф - классификацию болезней
    # на вход: список болезней (в формате списка []), используемая таблица, вид
    # вид - какой модуль функция будет использовать,
    # "b" --> bokeh
    # "c" --> ipycytoscape
    
    diseases = pd.DataFrame(diseases)
    diseases.columns = ["diseaseName"]
    table = table.merge(diseases, on = "diseaseName")
    G = networkx.from_pandas_edgelist(table, "diseaseName", "disease")
    if view == "b":
        # аутпут в ноутбук
        output_notebook()

        # подгонка размера и цвета
        degrees = dict(networkx.degree(G))
        networkx.set_node_attributes(G, name='degree', values=degrees)
        number_to_adjust_by = 5
        adjusted_node_size = dict([(node, 0.3*degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
        networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)
        size_by_this_attribute = 'adjusted_node_size'
        color_by_this_attribute = 'adjusted_node_size'

        # Blues8, Reds8, Purples8, Oranges8, Viridis8
        color_palette = Reds8

        title = 'Diseases'

        # hovering
        HOVER_TOOLTIPS = [("Type", "@index"), ("Degree", "@degree")]

        plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

        # network graph object
        network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

        # node sizes and colors according to node degree
        minimum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
        maximum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
        network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

        # edge opacity and width
        network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
        plot.renderers.append(network_graph)
        show(plot)
        
    if view == "c":
        classes_list = list(set(table["disease"]))
        diseases_list= list(set(table["diseaseName"]))

        my_style = [{'selector': 'node','style': {'font-family': 'arial','font-size': '10px','label': 'data(id)','background-color': 'red'}}]

        for item in classes_list:
            text = "node[id = \"" + item + "\"]"
            my_style.append({'selector': text,'style': {'font-family': 'arial','font-size': '10px','label': item,'background-color': 'green'}})
        for item in diseases_list:
            text = "node[id = \"" + item + "\"]"
            my_style.append({'selector': text,'style': {'font-family': 'arial','font-size': '10px','label': item,'background-color': 'blue'}})

        ipycytoscape_obj = ipycytoscape.CytoscapeWidget()
        ipycytoscape_obj.graph.add_graph_from_networkx(G)
        ipycytoscape_obj.set_style(my_style)
        ipycytoscape_obj.set_style(my_style)
        return ipycytoscape_obj